In [ ]:
import pandas as pd
import torch
from torch import optim, nn
from torchvision import models, transforms
import torchvision.transforms as T

In [ ]:
#https://towardsdatascience.com/image-feature-extraction-using-pytorch-e3b327c3607a
# df_train = pd.read_csv('./df_train_image.csv')
# df_test = pd.read_csv('./df_test_image.csv')
# df_train.drop(axis=1, columns=['classify_3'], inplace=True)
# df_test.drop(axis=1, columns=['classify_3'], inplace=True)

# df_train = pd.get_dummies(df_train, columns=['genre', 'source'])
# df_test = pd.get_dummies(df_test, columns=['genre', 'source'])

# df_test['genre_7'] = 0
# df_test['genre_3'] = 0
# df_test['genre_9'] = 0

out_features=10

# model = torchvision.models.resnet152(pretrained=False, num_classes = out_features)
vgg_model = models.vgg16(pretrained=True)
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
    self.pooling = model.avgpool
    self.flatten = nn.Flatten(start_dim=1, end_dim=-1)
    self.fc = model.classifier[0]
    self.fc1 = nn.Linear(4096, 10, bias=True) 
  
  def forward(self, x):
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    out = self.fc(out)
    out = self.fc1(out)
    return out 

new_model = FeatureExtractor(vgg_model)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

from tqdm import tqdm
import numpy as np

# Transform the image, so it becomes readable with the model
transform = T.Compose([
  T.ToPILImage(),
  T.CenterCrop(128),
  T.Resize(128),
  T.ToTensor()                              
])

# Will contain the feature
features = []

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
new_model

FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=

In [ ]:
# Iterate each image
for i in range(len(df_train['URL'])):
	# Set the image path
    path = df_train['URL'][i]
    img = cv2.imread(path)
	# Transform the image
    img = transform(img)
	# Reshape the image. PyTorch model reads 4-dimensional tensor
	# [batch_size, channels, width, height]
    img = img.reshape(1, 3, 128, 128)
    img = img.to(device)
	# We only extract features, so we don't need gradient
    with torch.no_grad():
		# Extract the feature from the image
        feature = new_model(img)
	# Convert to NumPy Array, Reshape it, and save it to features variable
    features.append(feature.cpu().detach().numpy().reshape(-1))

# Convert to NumPy Array
features = np.array(features)

In [ ]:
df_train_features = pd.DataFrame(features)
df_train_features.to_csv('./train_img_features_10.csv')

In [ ]:
# Iterate each image
test_features = []
for i in range(len(df_test['URL'])):
	# Set the image path
    path = df_test['URL'][i]
    img = cv2.imread(path)
	# Transform the image
    img = transform(img)
	# Reshape the image. PyTorch model reads 4-dimensional tensor
	# [batch_size, channels, width, height]
    img = img.reshape(1, 3, 128, 128)
    img = img.to(device)
	# We only extract features, so we don't need gradient
    with torch.no_grad():
		# Extract the feature from the image
        feature = new_model(img)
	# Convert to NumPy Array, Reshape it, and save it to features variable
    test_features.append(feature.cpu().detach().numpy().reshape(-1))

# Convert to NumPy Array
test_features = np.array(test_features)

In [ ]:
df_test_features = pd.DataFrame(test_features)
df_test_features.to_csv('./test_img_features_10.csv')